# Inferring
このレッスンでは、製品のレビューやニュース記事から、感情やトピックを推測する処理を学習します

## import

In [1]:
import openai
import json

## APIkey設定

In [2]:
json_file = open("./apikey.json", 'r')
api_key = json.load(json_file)
openai.api_key = api_key["apikey"]

## 実行関数

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

以下はランプ(製品)に関するレビューです

In [5]:
lamp_review = """
寝室用に素敵なランプが必要で、こちらは収納もついていて、価格帯も高くないので、購入しました。
早く手に入った。
ランプの紐が輸送中に切れてしまったのですが、新しいものを送ってくれました。
同じく数日以内に来ました。
組み立ても簡単でした。
私は部品が足りなかったので、彼らのサポートに連絡し、彼らは非常に迅速に不足している部品を手に入れました！
ルミナは、顧客と製品を大切にする素晴らしい会社のように思います！
"""

### レビューのpositive or negativeかを判定

In [6]:
prompt = f"""
次の製品レビューの評価を教えてください。
答えは「良」または「悪」で答えなさい

製品レビュー: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

良


### レビューの感情を予測しています

In [8]:
prompt = f"""
以下のレビューの書き手が表現している感情を5つリストアップしてください。
コンマで区切られた小文字の単語のリストとして、あなたの答えをフォーマットしてください。


製品レビュー : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

嬉しい, 満足, 安心, 感謝, 賞賛


### レビューの書き手が怒っているかを判定

In [9]:
prompt = f"""
以下のレビューの書き手は、怒りを表現しているかを答えてください。。
答えは「はい」「いいえ」のどちらかでお答えください。


製品レビュー : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

いいえ


### 製品名と会社名を抽出

In [10]:
prompt = f"""
以下のレビューから以下の項目を確認しなさい： 
- レビュアーが購入した商品
- 商品を作った会社名

「商品」と「会社名」をキーとするJSONオブジェクトとしてレスポンスをフォーマットしてください。
情報が存在しない場合は、「unknown」を値として使用しなさい。
回答はできるだけ短くしてください。
  


製品レビュー : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "商品": "寝室用ランプ",
  "会社名": "ルミナ"
}


### 複数タスクの同時実行

In [13]:
prompt = f"""
以下のレビューから以下の項目を確認しなさい： 
- 感情（肯定的または否定的）
- レビュアーは怒りを表現しているのか？(真or偽)
- レビュアーが購入した商品
- 商品を作った会社

回答は、「感情」「怒り」「商品」「会社」をキーとするJSONオブジェクトとしてフォーマットしてください。
情報が存在しない場合は、値として「unknown」を使用します。
回答はできるだけ短くしてください。
感情は「良」or「悪」のどちらかでフォーマットしてください。
怒りの値をブール値としてフォーマットしてください。

製品レビュー : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "感情": "良",
  "怒り": false,
  "商品": "寝室用ランプ",
  "会社": "ルミナ"
}


### 何の内容について記載されているかを推論

In [17]:
story = """
最近、政府が行ったアンケートでは 
公務員を対象に、勤務する部署に対する満足度を調査しました。
その結果、NASAが95％の満足度を得て、最も人気のある部署であることが判明しました。

NASAの職員の一人、ジョン・スミスは、この調査結果について次のようにコメントしています。
「NASAがトップになったことは驚きではありません。
素晴らしい人々と素晴らしい機会に恵まれた、素晴らしい職場だからです。
このような革新的な組織の一員であることを誇りに思います」と述べています。

この結果は、NASAの経営陣も歓迎しています。 
トム・ジョンソン所長は、「従業員がNASAでの仕事に満足していると聞いて感激している」と述べています。
私たちには、目標達成のためにたゆまぬ努力を続ける、才能豊かで献身的なチームがいます。
彼らの努力が報われたことを確認できて、とてもうれしいです。

また、今回の調査では、以下のことが明らかになりました。
社会保障庁は最も満足度が低く、仕事に満足していると回答した職員は45％にとどまりました。
政府は、この調査で従業員から寄せられた懸念に対処し、すべての部署で仕事満足度の向上に取り組むことを約束しました。
"""

In [18]:
prompt = f"""
次のニュースの内容に記載されている5つの話題を答えなさい。
各話題を1,2文程度にしなさい。
回答は、カンマで区切られた項目のリストとしてフォーマットしてください。

ニュース : '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. 政府が公務員の部署満足度を調査した結果、NASAが最も人気のある部署であることが判明した。
2. NASAの職員は、調査結果に満足し、素晴らしい職場であるとコメントした。
3. NASAの経営陣も調査結果を歓迎し、従業員の努力が報われたことを確認した。
4. 社会保障庁は最も満足度が低く、仕事に満足している職員は45％にとどまった。
5. 政府は、調査結果に基づき、すべての部署で仕事満足度の向上に取り組むことを約束した。


たとえば、複数の話題が混在した文章の場合
どれがNASAに関係する話題かを識別したい場合がある

In [22]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

prompt = f"""
以下の話題のリストの各項目が、以下の本文の話題であるかどうかを判断しなさい。

各話題について、0または1のリストで回答してください。
話題の配列:  {", ".join(topic_list)}

ニュース : '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 1
engineering: 0
employee satisfaction: 1
federal government: 1


In [23]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!
